In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from utils.data_extractor import download_single_ticker_forex_data
from utils.trainable_df import CreateTrainableDf

In [2]:
ticker1 = 'EUR'
ticker2 = 'INR'

In [3]:
eur_to_inr = download_single_ticker_forex_data(ticker1, ticker2)

/Users/arjunprakashrao/Drive/projects/forex-agent/utils/data_extractor.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  forex_data = yf.download(ticker_full, start='2000-01-02', end='2025-10-01')
[*********************100%***********************]  1 of 1 completed


In [4]:
eur_to_inr

,EUR_INR
Date,
2003-12-01,54.700001
2003-12-02,55.139000
2003-12-03,55.181000
2003-12-04,55.000999
2003-12-05,55.370998
...,...
2025-09-01,101.847000
2025-09-02,103.022003
2025-09-03,102.294998


In [5]:
config = {
    "lag":15,
    "price_col":f"{ticker1}_{ticker2}"
}

In [6]:
trainable_df = CreateTrainableDf(exchange_df=eur_to_inr, config=config)
trainable_df.create_trainable_df()
eur_to_inr = trainable_df.df

In [7]:
eur_to_inr

,EUR_INR,EUR_INR_lag_1,EUR_INR_lag_2,EUR_INR_lag_3,EUR_INR_lag_4,EUR_INR_lag_5,EUR_INR_lag_6,EUR_INR_lag_7,EUR_INR_lag_8,EUR_INR_lag_9,EUR_INR_lag_10,EUR_INR_lag_11,EUR_INR_lag_12,EUR_INR_lag_13,EUR_INR_lag_14,EUR_INR_lag_15,target
Date,,,,,,,,,,,,,,,,,
2003-12-22,56.346001,56.169998,56.441002,56.264000,56.025002,55.994999,55.779999,55.541000,55.291000,55.653999,55.563999,55.370998,55.000999,55.181000,55.139000,54.700001,0
2003-12-23,56.265999,56.346001,56.169998,56.441002,56.264000,56.025002,55.994999,55.779999,55.541000,55.291000,55.653999,55.563999,55.370998,55.000999,55.181000,55.139000,1
2003-12-24,56.604000,56.265999,56.346001,56.169998,56.441002,56.264000,56.025002,55.994999,55.779999,55.541000,55.291000,55.653999,55.563999,55.370998,55.000999,55.181000,0
2003-12-25,56.584999,56.604000,56.265999,56.346001,56.169998,56.441002,56.264000,56.025002,55.994999,55.779999,55.541000,55.291000,55.653999,55.563999,55.370998,55.000999,1
2003-12-26,56.455002,56.584999,56.604000,56.265999,56.346001,56.169998,56.441002,56.264000,56.025002,55.994999,55.779999,55.541000,55.291000,55.653999,55.563999,55.370998,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-01,101.847000,102.233002,102.009003,101.920998,101.669998,100.411003,101.300003,101.306999,101.299004,101.774002,101.202003,102.083000,102.345001,102.260002,101.760002,101.751999,1
2025-09-02,103.022003,101.847000,102.233002,102.009003,101.920998,101.669998,100.411003,101.300003,101.306999,101.299004,101.774002,101.202003,102.083000,102.345001,102.260002,101.760002,0
2025-09-03,102.294998,103.022003,101.847000,102.233002,102.009003,101.920998,101.669998,100.411003,101.300003,101.306999,101.299004,101.774002,101.202003,102.083000,102.345001,102.260002,1


# Graphs

In [8]:
eur_to_inr.sum(), eur_to_inr.mean(), eur_to_inr.std()

(EUR_INR           411733.690670
 EUR_INR_lag_1     411686.405972
 EUR_INR_lag_2     411640.232975
 EUR_INR_lag_3     411594.201977
 EUR_INR_lag_4     411547.204975
 EUR_INR_lag_5     411501.352974
 EUR_INR_lag_6     411454.899971
 EUR_INR_lag_7     411408.431969
 EUR_INR_lag_8     411361.801971
 EUR_INR_lag_9     411315.785973
 EUR_INR_lag_10    411270.938969
 EUR_INR_lag_11    411225.009964
 EUR_INR_lag_12    411178.703964
 EUR_INR_lag_13    411132.585960
 EUR_INR_lag_14    411085.950958
 EUR_INR_lag_15    411039.448956
 target              2766.000000
 dtype: float64,
 EUR_INR           73.106124
 EUR_INR_lag_1     73.097728
 EUR_INR_lag_2     73.089530
 EUR_INR_lag_3     73.081357
 EUR_INR_lag_4     73.073012
 EUR_INR_lag_5     73.064871
 EUR_INR_lag_6     73.056623
 EUR_INR_lag_7     73.048372
 EUR_INR_lag_8     73.040093
 EUR_INR_lag_9     73.031922
 EUR_INR_lag_10    73.023959
 EUR_INR_lag_11    73.015804
 EUR_INR_lag_12    73.007582
 EUR_INR_lag_13    72.999394
 EUR_INR_lag_14 

# Time series analysis

In [9]:
from statsmodels.tsa.stattools import adfuller
from models.time_series import time_series as ts

In [10]:
# Using ADF test to check for stationarity
eur_to_inr_list = eur_to_inr[config['price_col']].to_list()
ad_stats = adfuller(eur_to_inr_list, regression='ct', autolag='BIC')
ad_stats

(np.float64(-3.3858584939785965),
 np.float64(0.05330619447690662),
 5,
 5626,
 {'1%': np.float64(-3.9603800528273756),
  '5%': np.float64(-3.411270662557412),
  '10%': np.float64(-3.1275097046499853)},
 np.float64(7974.992643754999))

In [11]:
forecast = ts.arima_train_and_pred(eur_to_inr[config['price_col']].to_numpy(), pred_period=1)

/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [12]:
eur_to_inr[config['price_col']].to_numpy()

array([ 56.34600067,  56.26599884,  56.60400009, ..., 102.29499817,
       102.61399841, 103.45469666], shape=(5632,))

In [13]:
forecast

1

In [14]:
eur_to_inr.iloc[-365]

EUR_INR           90.268997
EUR_INR_lag_1     90.366997
EUR_INR_lag_2     89.237602
EUR_INR_lag_3     90.331001
EUR_INR_lag_4     90.117996
EUR_INR_lag_5     89.662003
EUR_INR_lag_6     88.919601
EUR_INR_lag_7     89.867996
EUR_INR_lag_8     89.863998
EUR_INR_lag_9     90.065002
EUR_INR_lag_10    90.101997
EUR_INR_lag_11    90.328003
EUR_INR_lag_12    90.252998
EUR_INR_lag_13    90.341003
EUR_INR_lag_14    90.712997
EUR_INR_lag_15    90.128998
target             1.000000
Name: 2024-04-10 00:00:00, dtype: float64

In [15]:
trainable_df.backtest(ts.arima_train_and_pred)

/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimizat

Accuracy Score: 0.4794520547945205


/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/arjunprakashrao/Drive/projects/forex-agent/.venv/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<function sklearn.metrics._classification.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)>